# pymars Real Data Example: Pima Indians Diabetes Dataset

This notebook demonstrates the use of pymars on real health data using the Pima Indians Diabetes dataset. This dataset is commonly used in machine learning for predicting the onset of diabetes based on diagnostic measurements.

## Dataset Information

The dataset contains 768 observations with the following features:
- Number of times pregnant
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- Diastolic blood pressure (mm Hg)
- Triceps skin fold thickness (mm)
- 2-Hour serum insulin (mu U/ml)
- Body mass index (weight in kg/(height in m)^2)
- Diabetes pedigree function
- Age (years)
- Class variable (0 or 1)

This example demonstrates how pymars can be used for health economic outcomes research, particularly in modeling complex relationships between health indicators and disease outcomes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import pymars as earth

# For reproducibility
np.random.seed(42)

## Load and Explore the Data

In [ ]:
# Load the dataset
column_names = [
    'pregnancies', 'glucose', 'blood_pressure', 'skin_thickness',
    'insulin', 'bmi', 'diabetes_pedigree', 'age', 'outcome'
]

data = pd.read_csv('pima-indians-diabetes.csv', names=column_names)

print(f"Dataset shape: {data.shape}")
print(f"\nFirst few rows:")
print(data.head())

print(f"\nDataset info:")
print(data.info())

In [ ]:
# Check for missing values (represented as 0 in this dataset)
print("Zero values per column (potential missing data):")
zero_counts = (data == 0).sum()
print(zero_counts)

# Basic statistics
print(f"\nBasic statistics:")
print(data.describe())

# Outcome distribution
print(f"\nOutcome distribution:")
print(data['outcome'].value_counts())
print(f"Percentage with diabetes: {data['outcome'].mean()*100:.2f}%")

In [ ]:
# Visualize the data
plt.figure(figsize=(15, 10))

# Distribution of features by outcome
features = ['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree', 'age']

for i, feature in enumerate(features, 1):
    plt.subplot(2, 4, i)
    for outcome in [0, 1]:
        subset = data[data['outcome'] == outcome][feature]
        plt.hist(subset, bins=30, alpha=0.7, label=f'Diabetes: {"Yes" if outcome else "No"}', density=True)
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()
    plt.title(f'{feature} Distribution')

plt.tight_layout()
plt.show()

## Data Preprocessing

In [ ]:
# Handle zero values as missing data (except for pregnancies and outcome)
data_processed = data.copy()

# Columns where 0 represents missing data
cols_with_zeros = ['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi']

# Replace 0s with NaN for these columns
for col in cols_with_zeros:
    data_processed[col] = data_processed[col].replace(0, np.nan)

# Check missing data
print("Missing data after replacement:")
print(data_processed.isnull().sum())

# Separate features and target
X = data_processed.drop('outcome', axis=1)
y = data_processed['outcome']

print(f"\nFinal dataset shape: X={X.shape}, y={y.shape}")

## Fit MARS Model for Regression

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Fit MARS model for regression (predicting probability)
print("\nFitting MARS regression model...")
mars_reg = earth.Earth(
    max_degree=2,      # Allow two-way interactions
    penalty=3.0,       # GCV penalty
    max_terms=21,     # Max number of terms
    minspan_alpha=0.05,  # Minimum span control
    endspan_alpha=0.05,  # End span control
    allow_linear=True,   # Allow linear terms
    allow_missing=True,  # Handle missing values
    feature_importance_type='gcv'  # Calculate feature importance
)

# Fit the model
mars_reg.fit(X_train.values, y_train.values)

# Model evaluation
train_r2 = mars_reg.score(X_train.values, y_train.values)
test_r2 = mars_reg.score(X_test.values, y_test.values)

print(f"\nMARS Regression Model Summary:")
print(f"Number of basis functions: {len(mars_reg.basis_) - 1}")  # Subtract 1 for intercept
print(f"Training R-squared: {train_r2:.3f}")
print(f"Test R-squared: {test_r2:.3f}")
print(f"GCV Score: {mars_reg.gcv_:.3f}")

In [ ]:
# Feature importance
print(f"\nFeature Importances:")
feature_names = list(X.columns)
importances = mars_reg.feature_importances_

for i, (name, imp) in enumerate(zip(feature_names, importances)):
    print(f"  {name}: {imp:.4f}")

## MARS for Classification

In [ ]:
# Fit MARS model for classification
print("\nFitting MARS classification model...")
mars_clf = earth.EarthClassifier(
    max_degree=2,      # Allow two-way interactions
    penalty=3.0,       # GCV penalty
    max_terms=21,     # Max number of terms
    minspan_alpha=0.05,  # Minimum span control
    endspan_alpha=0.05,  # End span control
    allow_linear=True,   # Allow linear terms
    allow_missing=True   # Handle missing values
)

# Fit the model
mars_clf.fit(X_train.values, y_train.values)

# Make predictions
y_pred = mars_clf.predict(X_test.values)
y_pred_proba = mars_clf.predict_proba(X_test.values)[:, 1]

# Model evaluation
accuracy = mars_clf.score(X_test.values, y_test.values)
auc_score = roc_auc_score(y_test.values, y_pred_proba)

print(f"\nMARS Classification Model Summary:")
print(f"Accuracy: {accuracy:.3f}")
print(f"AUC Score: {auc_score:.3f}")

# Classification report
print(f"\nClassification Report:")
print(classification_report(y_test.values, y_pred))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test.values, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Diabetes', 'Diabetes'],
            yticklabels=['No Diabetes', 'Diabetes'])
plt.title('Confusion Matrix - MARS Classification')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test.values, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - MARS Classification')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

## Interpretability and Model Explanation

In [ ]:
# Show selected basis functions
print("Selected Basis Functions:")
for i, (bf, coef) in enumerate(zip(mars_reg.basis_, mars_reg.coef_)):
    if i == 0:  # Intercept
        print(f"  {bf}: {coef:.3f}")
    else:
        print(f"  {bf}: {coef:.3f}")

In [ ]:
# Generate partial dependence plots for top features
top_features = np.argsort(importances)[-4:][::-1]
fig, axes = earth.plot_partial_dependence(
    mars_reg, 
    X_train.values, 
    top_features, 
    feature_names=feature_names, 
    n_cols=2, 
    figsize=(12, 10)
)
plt.suptitle("Partial Dependence Plots for Top 4 Features", fontsize=16)
plt.tight_layout()
plt.show()

## Comparison with Other Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Impute missing values for other models
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Define models
models = {
    'MARS': mars_clf,
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(probability=True, random_state=42)
}

# Fit and evaluate models
results = {}
for name, model in models.items():
    if name == 'MARS':
        # MARS handles missing values internally
        model.fit(X_train.values, y_train.values)
        y_pred = model.predict(X_test.values)
        y_pred_proba = model.predict_proba(X_test.values)[:, 1]
    else:
        # Other models need imputed data
        if name == 'Logistic Regression' or name == 'SVM':
            model.fit(X_train_scaled, y_train.values)
            y_pred = model.predict(X_test_scaled)
            y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
        else:
            model.fit(X_train_imputed, y_train.values)
            y_pred = model.predict(X_test_imputed)
            y_pred_proba = model.predict_proba(X_test_imputed)[:, 1]
    
    accuracy = np.mean(y_pred == y_test.values)
    auc = roc_auc_score(y_test.values, y_pred_proba)
    results[name] = {'accuracy': accuracy, 'auc': auc}
    
    print(f"{name} - Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

In [ ]:
# Visualize comparison
model_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in model_names]
aucs = [results[name]['auc'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, accuracies, width, label='Accuracy')
bars2 = ax.bar(x + width/2, aucs, width, label='AUC')

ax.set_xlabel('Models')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(model_names)
ax.legend()
ax.set_ylim([0, 1])

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Feature Importance Analysis

In [ ]:
# Compare feature importances across models
plt.figure(figsize=(12, 8))

# MARS feature importance
plt.subplot(2, 2, 1)
plt.barh(range(len(importances)), importances)
plt.yticks(range(len(importances)), feature_names)
plt.xlabel('Importance')
plt.title('MARS Feature Importance')
plt.gca().invert_yaxis()

# Random Forest feature importance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_imputed, y_train.values)
rf_importances = rf_model.feature_importances_

plt.subplot(2, 2, 2)
plt.barh(range(len(rf_importances)), rf_importances)
plt.yticks(range(len(rf_importances)), feature_names)
plt.xlabel('Importance')
plt.title('Random Forest Feature Importance')
plt.gca().invert_yaxis()

# Logistic Regression coefficients
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train.values)
lr_coefs = np.abs(lr_model.coef_[0])

plt.subplot(2, 2, 3)
plt.barh(range(len(lr_coefs)), lr_coefs)
plt.yticks(range(len(lr_coefs)), feature_names)
plt.xlabel('|Coefficient|')
plt.title('Logistic Regression Feature Importance')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

## Health Economics Application Example

Let's demonstrate how pymars can be applied to health economics research using this dataset:

In [ ]:
# Simulate healthcare costs based on the diabetes outcome and other factors
# This demonstrates how pymars can be used in health economic outcomes research

def simulate_healthcare_costs(row):
    """Simulate healthcare costs based on diabetes status and other health factors"""
    base_cost = 2000  # Base annual healthcare cost
    
    # Cost associated with diabetes
    diabetes_cost = 8000 * row['outcome']  # Extra $8000 for diabetic patients
    
    # Cost associated with BMI (obesity-related costs)
    bmi_cost = 0
    if not np.isnan(row['bmi']):
        if row['bmi'] >= 30:  # Obese
            bmi_cost = 3000
        elif row['bmi'] >= 25:  # Overweight
            bmi_cost = 1500
    
    # Cost associated with age
    age_cost = 50 * max(0, row['age'] - 40)  # Increasing costs after age 40
    
    # Cost associated with blood pressure
    bp_cost = 0
    if not np.isnan(row['blood_pressure']):
        if row['blood_pressure'] >= 90:  # Hypertensive
            bp_cost = 2000
    
    # Random variation
    random_variation = np.random.normal(0, 1000)
    
    # Total cost
    total_cost = base_cost + diabetes_cost + bmi_cost + age_cost + bp_cost + random_variation
    
    return max(0, total_cost)  # Ensure non-negative costs

# Apply the cost simulation function
healthcare_costs = data.apply(simulate_healthcare_costs, axis=1)

print(f"Simulated healthcare costs:")
print(f"  Mean cost: ${healthcare_costs.mean():,.2f}")
print(f"  Median cost: ${healthcare_costs.median():,.2f}")
print(f"  Min cost: ${healthcare_costs.min():,.2f}")
print(f"  Max cost: ${healthcare_costs.max():,.2f}")

# Compare costs by diabetes status
costs_by_diabetes = healthcare_costs.groupby(data['outcome']).describe()
print(f"\nCosts by diabetes status:")
print(costs_by_diabetes)

In [ ]:
# Fit MARS model to predict healthcare costs
print("\nFitting MARS model to predict healthcare costs...")

# Combine original features with the outcome for cost prediction
X_cost = data.drop('outcome', axis=1).copy()
X_cost['diabetes'] = data['outcome']  # Include diabetes status as a predictor
y_cost = healthcare_costs

# Split the data
X_cost_train, X_cost_test, y_cost_train, y_cost_test = train_test_split(
    X_cost, y_cost, test_size=0.2, random_state=42
)

# Fit MARS model for cost prediction
mars_cost = earth.Earth(
    max_degree=2,      # Allow two-way interactions
    penalty=3.0,       # GCV penalty
    max_terms=21,     # Max number of terms
    minspan_alpha=0.05,  # Minimum span control
    endspan_alpha=0.05,  # End span control
    allow_linear=True,   # Allow linear terms
    allow_missing=True,  # Handle missing values
    feature_importance_type='gcv'  # Calculate feature importance
)

# Fit the model
mars_cost.fit(X_cost_train.values, y_cost_train.values)

# Model evaluation
train_r2_cost = mars_cost.score(X_cost_train.values, y_cost_train.values)
test_r2_cost = mars_cost.score(X_cost_test.values, y_cost_test.values)

print(f"\nHealthcare Cost Prediction Model Summary:")
print(f"Number of basis functions: {len(mars_cost.basis_) - 1}")  # Subtract 1 for intercept
print(f"Training R-squared: {train_r2_cost:.3f}")
print(f"Test R-squared: {test_r2_cost:.3f}")
print(f"GCV Score: {mars_cost.gcv_:.3f}")

In [ ]:
# Feature importance for cost prediction
cost_feature_names = list(X_cost.columns)
cost_importances = mars_cost.feature_importances_

print(f"\nFeature Importances for Healthcare Cost Prediction:")
for name, imp in zip(cost_feature_names, cost_importances):
    print(f"  {name}: {imp:.4f}")

# Visualize feature importances
plt.figure(figsize=(10, 6))
plt.barh(range(len(cost_importances)), cost_importances)
plt.yticks(range(len(cost_importances)), cost_feature_names)
plt.xlabel('Importance')
plt.title('Feature Importance for Healthcare Cost Prediction (MARS)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Generate partial dependence plots for cost prediction
top_cost_features = np.argsort(cost_importances)[-4:][::-1]
fig, axes = earth.plot_partial_dependence(
    mars_cost, 
    X_cost_train.values, 
    top_cost_features, 
    feature_names=cost_feature_names, 
    n_cols=2, 
    figsize=(12, 10)
)
plt.suptitle("Partial Dependence Plots for Healthcare Cost Prediction", fontsize=16)
plt.tight_layout()
plt.show()

## Conclusion

This example demonstrates how pymars can be applied to real health data for both clinical prediction and health economics research:

1. **Clinical Prediction**: Using MARS to predict diabetes onset based on diagnostic measurements
2. **Health Economics**: Using MARS to model healthcare costs based on patient characteristics
3. **Interpretability**: Leveraging MARS's explicit basis functions for policy-relevant insights
4. **Missing Data Handling**: Demonstrating pymars's ability to handle missing values in health data
5. **Non-linear Relationships**: Capturing complex interactions between health factors

The Pima Indians Diabetes dataset is an excellent example for showcasing pymars capabilities because it contains:
- Multiple continuous variables with potential non-linear relationships
- Missing data patterns common in health records
- Clear clinical outcomes for prediction
- Opportunities for health economic modeling

This real-world example demonstrates how pymars can be valuable for health economic outcomes research, where understanding complex relationships between patient characteristics, health outcomes, and costs is crucial for evidence-based policy making.